https://llama-cpp-python.readthedocs.io/en/latest/

In [2]:
from llama_cpp import Llama
max_tokens = 32000
# llama_llm = Llama(model_path="/Volumes/4Tera_SSD_2022/llm_models/llama-2-7b-chat.Q5_K_M.gguf", chat_format="llama-2")
model_file_path = "/Users/jmcaffee/llm_models/Llama-3-8B-Instruct-32k-v0.1.Q4_K_M.gguf"
llm = Llama(model_path=model_file_path, chat_format="llama-3", n_ctx=max_tokens)
llm.verbose = False # don't print a bunch of stuff
# llm = Llama(model_path="/Volumes/4Tera_SSD_2022/llm_models/llama-2-13b-chat.Q5_K_M.gguf", chat_format="llama-2", n_ctx=2048) # n_ctx=2048
# function_llm = Llama(model_path="/Volumes/4Tera_SSD_2022/llm_models/functionary-7b-v1.Q5_K.gguf", chat_format="llama-2") # n_ctx=2048

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /Users/jmcaffee/llm_models/Llama-3-8B-Instruct-32k-v0.1.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models--MaziyarPanahi--Llama-3-8B-Ins...
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 32000
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model

In [3]:
import time
import json

def prompt(question):
    output = llm(
        f"Q: {question} A:", # Prompt
        # max_tokens=512,
        max_tokens=max_tokens,
        stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
        echo=False, # Echo the prompt back in the output
    )
    answer = output['choices'][0]['text']
    print(f"Q: {question} \nA:{answer}")
    return answer

def prompt2(question):
    start_time_seconds = time.time()
    output = llm.create_chat_completion(
        max_tokens=max_tokens,
        messages = [
            {
                "role": "system",
                "content": "You are an assistant who perfectly responds to questions with as many details as possible, without making up facts."
            },
            {
                "role": "user",
                "content": question
            }
        ]
    )
    elapsed_time_seconds = time.time() - start_time_seconds
    # print(output)
    answer = output['choices'][0]['message']['content']
    print(f'answer generated in {elapsed_time_seconds} seconds')
    print(f"Q: {question} \nA:{answer}")
    return answer

def stream_prompt(question):
    start_time_seconds = time.time()
    stream = llm.create_chat_completion(
        max_tokens=max_tokens,
        messages = [
            {
                "role": "system",
                "content": "You are an assistant who perfectly responds to questions with as many details as possible, without making up facts."
            },
            {
                "role": "user",
                "content": question
            }
        ],
        stream = True,
    )


    for output in stream:
        choices = output['choices']
        choice = choices[0]
        delta = choice['delta']
        if 'content' in delta:
            print(delta['content'], end='')
    print()

    elapsed_time_seconds = time.time() - start_time_seconds
    print(f'answer generated in {elapsed_time_seconds} seconds')

In [4]:
stream_prompt("Who was Juan de Leon?")

Juan Sebastián Elcano was a Spanish explorer and navigator who is best known for being the first person to circumnavigate the globe in a single expedition. He was born on 1476 in Getaria, Spain, and his full name was Juan Sebastián Elcano y Hurtado de Mendoza.

Elcano joined Christopher Columbus's second voyage to the Americas in 1493 as a young man and later became a member of the crew of the expedition led by Vasco Núñez de Balboa, who explored the Pacific coast of South America. In 1519, he was chosen by King Charles I of Spain to join an expedition led by Ferdinand Magellan, a Portuguese explorer, to reach Asia by sailing west across the Atlantic Ocean.

Magellan's expedition set sail from Sanlúcar de Barrameda in September 1519 with five ships and over 200 men. Elcano became the pilot-major of the expedition, which aimed to find a new route to the East Indies. The journey was marked by many challenges, including mutinies, storms, and scurvy, but they eventually reached the Pacific

In [5]:
from projects.utils.with_time import with_time

@with_time
def get_n_words_from_text(text:str, number_of_words=1000):
    words = text.split()
    first_n_words = ' '.join(words[:number_of_words])
    return first_n_words

https://archive.org/stream/MajjhimaNikaya/MajjhimaNikayaSinglePaged._djvu.txt

In [13]:
with open('../../data/buddhist-texts/majjhima nikaya.txt', 'r') as file:
    large_text = file.read()
    
large_text = get_n_words_from_text(large_text, number_of_words=21000)    
stream_prompt(f"""
Using the text below, which is surrounded by <text> tags, summarize what is being discussed.  Include summaries of each sutta, the key points discussed, and then provide a summary of all the suttas, highlighting the key concepts and terms
<text>
{large_text}
</text>
""")    

get_n_words_from_text took 2.284121513366699e-05 ms to complete
The text appears to be a collection of Buddhist sutras or discourses from the Pali Canon, specifically from the Majjhima Nikaya (Middle Length Discourses) and the Digha Nikaya (Longer Discourses). The text consists of nine suttas (discourses) that discuss various topics related to the nature of reality, the path to enlightenment, and the characteristics of a noble disciple. Here is a summary of each sutta:
1. Mulapariyaya Sutta: This sutta discusses the root of all things, including the tendency to conceive oneself as earth, water, fire, air, beings, gods, and other phenomena, and how this conception leads to suffering.
2. Sabbasava Sutta: This sutta explains the importance of abandoning various taints (defilements) such as greed, hatred, delusion, and ignorance through seeing, restraining, using, enduring, avoiding, removing, and developing wholesome states.
3. Dhammadayada Sutta: This sutta emphasizes the importance of b

In [8]:
text = get_n_words_from_text(large_text, number_of_words=10000) 
print(text)

get_n_words_from_text took 9.279251098632812e-07 ms to complete
Part One The Root Fifty Discourses (MulapannasapaH) 1 The Division of the Discourse on the Root (Mulapariyayavagga) 1 Mulapariyaya Sutta The Root of All Things [1] 1. Thus have I heard. 1 On one occasion the Blessed One was living in Ukkattha in the Subhaga Grove at the root of a royal sala tree. There he addressed the bhikkhus thus: "Bhikkhus." 2 - "Venerable sir," they replied. The Blessed One said this: 2. "Bhikkhus, I shall teach you a discourse on the root of all things. 3 Listen and attend closely to what I shall say." - "Yes, venerable sir," the bhikkhus replied. The Blessed One said this: (the ordinary person) 3. "Here, bhikkhus, an untaught ordinary person, 4 who has no regard for noble ones and is unskilled and undisciplined in their Dhamma, who has no regard for true men and is unskilled and undisciplined in their Dhamma, perceives earth as earth. 5 Having perceived earth as earth, he conceives [himself as] eart